# Code zu der Bachelorarbeit:
# "Comparitve Study von Machine Learning Modellen zur Erkennung von Web Schwachstellen"
## von Nils Pudenz, 2735230

# Importe

In [23]:

%pip install kaggle scikit-learn xgboost catboost tabpfn pandas numpy matplotlib seaborn -q

Note: you may need to restart the kernel to use updated packages.


In [24]:
import os
import zipfile
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (precision_score, recall_score, f1_score,
                             confusion_matrix)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from tabpfn import TabPFNClassifier
import matplotlib.pyplot as plt

In [25]:
#Deterministische Ausgabe generieren, um die Reproduzierbarkeit zu gewährleisten
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## Dowload Kaggle Datasets
Requires Kaggle API credentials ('~/.kaggle/kaggle.json') für API-Token, um zugriff auf die Datenbanken über das Kaggle Konto zu bekommen

In [26]:
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

In [27]:
#Dowload der Datasets von Kaggle
os.system("kaggle datasets download -d syedsaqlainhussain/sql-injection-dataset -p data --unzip --quiet")
os.system("kaggle datasets download -d syedsaqlainhussain/cross-site-scripting-xss-dataset-for-deep-learning -p data --unzip --quiet")


0

## Load and Inspect Data

In [28]:
SQL_CSV = next(DATA_DIR.glob("**/sql*/*.csv"), None) or next(DATA_DIR.glob("**/*SQL*.csv"), None)
XSS_CSV = next(DATA_DIR.glob("**/xss*/*.csv"), None) or next(DATA_DIR.glob("**/*XSS*.csv"), None)


In [ ]:
#csv to dataframe
sql_df = pd.read_csv(SQL_CSV, encoding="utf-16", sep=",", low_memory=False) #utf-8 Fehler
xss_df = pd.read_csv(XSS_CSV)

In [30]:
for name, df in {"SQL": sql_df, "XSS": xss_df}.items():
    print(f"{name} dataset shape: {df.shape}")
    display(df.head())

SQL dataset shape: (4200, 2)


,Sentence,Label
0,a,1
1,a',1
2,a' --,1
3,a' or 1 = 1; --,1
4,@,1


XSS dataset shape: (13686, 3)


,Unnamed: 0,Sentence,Label
0,0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,2,"\t </span> <span class=""reference-text"">Steeri...",0
3,3,"\t </span> <span class=""reference-text""><cite ...",0
4,4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0
